In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import parse
import scipy.optimize
import pickle
from contextlib import redirect_stdout

import communicate

ROOT = Path('/Users/gerry/DropboxGatech/Painting/Graffiti_media+data/2022-09-15_slm')
fbase = '04_run'
duration = 2.5

In [ ]:
# Utils
def get_last_first_indices(t, signal, sensitivity=0.03, timebuffer=0.1):
    ilast = np.where(np.all(np.abs(np.diff(signal, axis=0)) > sensitivity, axis=1))[0][-1]
    tlast = t[ilast] + timebuffer
    ilast = np.argmin(np.abs(t - tlast))
    ifirst = np.argmin(np.abs(t - (tlast - duration)))
    return ifirst, ilast

In [ ]:
# Load log data
with open(ROOT / f'{fbase}.txt', 'r') as f:
    lines = f.readlines()
with redirect_stdout(open('/dev/null', 'w')):
    lines = [communicate.parse_line(line) for line in lines]
log = {'ts': [], 'xys': [], 'ls': []}
for cdata, mdata, sdata, cstate, mstates in lines:
    if cstate is None:
        continue
    log['ts'].append(cstate.time_us / 1e6)
    log['xys'].append([cstate.cur_x, cstate.cur_y])
    log['ls'].append([mstate.length for mstate in mstates])
log = {k: np.array(v) for k, v in log.items()}

# Fix log timestamps
bad = np.where(np.diff(log['ts']) < 0)[0]
for i in bad:
    log['ts'][i + 1:] += 10
log['ts'] -= log['ts'][0]

s,e = get_last_first_indices(log['ts'], log['xys'], sensitivity=0.01, timebuffer=0.1)
log['ts'] = log['ts'][s:e]
log['xys'] = log['xys'][s:e]
log['ls'] = log['ls'][s:e]
plt.plot(log['ts'], log['xys'])

In [ ]:
# Load mocap data
data = pd.read_csv(ROOT / f'{fbase}.csv', skiprows=1)

get_xyz = lambda name: data.loc[4:, data.loc[0] == name].values.astype(float)
ees_ = get_xyz('ee')
frames = [get_xyz(i) for i in '0123']

In [ ]:
frame = np.array([np.nanmean(corner, axis=0) for corner in frames])
ees = ees_ - frame[3]
frame = frame - frame[3]
mocap_ls = ees.reshape(-1, 3, 1) - frame.T.reshape(1, 3, 4)
mocap_ls[:, 0, :] = 0
mocap_ls = np.sqrt(np.sum(np.square(mocap_ls), axis=1))
mocap_ts = np.arange(0, mocap_ls.shape[0]) / 120
print(mocap_ls.shape)

s,e = get_last_first_indices(mocap_ts, ees, sensitivity=0.001, timebuffer=0.1)
ees = ees[s:e, [2, 1]]
mocap_ts = mocap_ts[s:e]
plt.plot(mocap_ts, ees)

In [ ]:
# Load SLM data
with open(f'/Users/gerry/GIT_REPOS/art_skills/python/src/trajectories/{fbase}.p', 'rb') as f:
    slm = pickle.load(f)
input_stroke = slm[-1][0]
estimated_output = slm[-1][1]
output_stroke = []
for _, new_output in slm:
    output_stroke.extend(new_output[len(output_stroke):])
output_stroke = np.array(output_stroke)
print(input_stroke.shape, estimated_output.shape, output_stroke.shape)

In [ ]:
# Scale SLM data according to the same scaling used in realtime
width = 2.96
def scale(txy):
    ret = txy * 1
    ret[:, 1] *= width
    ret[:, 2] = (0.64 - ret[:, 2] + 0.1) * width
    return ret
input_stroke_ = scale(input_stroke)
estimated_output_ = scale(estimated_output)
output_stroke_ = scale(output_stroke)

In [ ]:
# Plot
plt.plot(*input_stroke_[:, 1:].T, 'k', label='input')
plt.plot(*output_stroke_[:, 1:].T, 'r', label='output')
plt.plot(*estimated_output_[:, 1:].T, 'g', label='final estimate')
plt.plot(*ees.T, 'k.', label='mocap')
plt.legend()
plt.axis('equal')
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.title('Real-time Sigma Lognormal Model Fitting and Execution on a Cable Robot (Single-Stroke)')